# Days Hospitalized - ML Modeling

------------
### Data Vectorization

In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.sql.functions import *

In [4]:
dataset = spark.table("mldata_days_hospitalized")
# display(dataset)

Patient,dataset,days_hospitalized,City,Marital,Race,Ethnicity,Gender,Suffix,County,State,Revenue,Healthcare_Coverage,pct_covered,age_years,allergy_232347008,allergy_232350006,allergy_300913006,allergy_300916003,allergy_417532002,allergy_418689008,allergy_419263009,allergy_419474003,allergy_420174000,allergy_424213003,allergy_425525006,allergy_714035009,allergy_91930004,allergy_91934008,allergy_91935009,careplan_133899007,careplan_133901003,careplan_134435003,careplan_170836005,careplan_182964004,careplan_183401008,careplan_225358003,careplan_384758001,careplan_385691007,careplan_386257007,careplan_386522008,careplan_395082007,careplan_408869004,careplan_412776001,careplan_443402002,careplan_47387005,careplan_53950000,careplan_698360004,careplan_699728000,careplan_703040004,careplan_711282006,careplan_718347000,careplan_734163000,careplan_735984001,careplan_736252007,careplan_736254008,careplan_736285004,careplan_736353004,careplan_736690008,careplan_737434004,careplan_737471002,careplan_75162002,careplan_781831000000109,careplan_869761000000107,careplan_91251008,condition_10509002,condition_109838007,condition_110030002,condition_11218009,condition_124171000119105,condition_126906006,condition_127013003,condition_127295002,condition_128613002,condition_1501000119109,condition_1551000119108,condition_156073000,condition_157141000119108,condition_15724005,condition_15777000,condition_16114001,condition_161621004,condition_161622006,condition_162573006,condition_162864005,condition_1734006,condition_185086009,condition_190905008,condition_19169002,condition_192127007,condition_195662009,condition_195967001,condition_196416002,condition_197927001,condition_198992004,condition_200936003,condition_201834006,condition_206523001,condition_22298006,condition_225444004,condition_230265002,condition_230690007,condition_232353008,condition_233604007,condition_233678006,condition_235919008,condition_236077008,condition_237602007,condition_239720000,condition_239872002,condition_239873007,condition_24079001,condition_241929008,condition_254632001,condition_254637007,condition_254837009,condition_262574004,condition_263102004,condition_26929004,condition_271737000,condition_275272006,condition_283371005,condition_283385000,condition_284549007,condition_284551006,condition_287185009,condition_301011002,condition_302870006,condition_307731004,condition_30832001,condition_314994000,condition_33737001,condition_359817006,condition_35999006,condition_363406005,condition_367498001,condition_368581000119106,condition_36923009,condition_36971009,condition_370143000,condition_370247008,condition_38822007,condition_398254007,condition_39848009,condition_399211009,condition_40055000,condition_40275004,condition_403190006,condition_403191005,condition_403192003,condition_406602003,condition_408512008,condition_410429000,condition_422034002,condition_424132000,condition_425048006,condition_427089005,condition_428251008,condition_429007001,condition_429280009,condition_431855005,condition_431856006,condition_433144002,condition_43878008,condition_44054006,condition_443165006,condition_444448004,condition_444470001,condition_44465007,condition_444814009,condition_446096008,condition_448417001,condition_448813005,condition_449868002,condition_45816000,condition_47200007,condition_47505003,condition_47693006,condition_48333001,condition_49436004,condition_53741008,condition_55680006,condition_55822004,condition_5602001,condition_58150001,condition_59621000,condition_6072007,condition_60951000119105,condition_62106007,condition_62564004,condition_64859006,condition_65275009,condition_65363002,condition_65710008,condition_65966004,condition_67811000119102,condition_68496003,condition_698423002,condition_698754002,condition_69896004,condition_703151001,condition_70704007,condition_707577004,condition_713197008,condition_7200002,condition_72892002,condition_74400008,condition_75498004,condition_79586000,condition_80394007,condition_82423001,condition_83664006,co

In [5]:
label = "days_hospitalized"
allColumns = dataset.columns

categoricalColumns = [
  "City",
  "Marital",
  "Race",
  "Ethnicity",
  "Gender",
  "Suffix",
  "County",
#   "State",
  "smoker_status"
]

ignoreColumns = ["Patient", "dataset", "State", label]
numericalColumns = list(set(allColumns) - set(categoricalColumns) - set(ignoreColumns))
categoricalColumnsclassVec = [c + "classVec" for c in categoricalColumns]

stages = []

In [6]:
for categoricalColumn in categoricalColumns:
  print(categoricalColumn)
  ## Category Indexing with StringIndexer
  stringIndexer = StringIndexer(inputCol=categoricalColumn, outputCol = categoricalColumn+"Index").setHandleInvalid("skip")
  ## Use OneHotEncoder to convert categorical variables into binary SparseVectors
  encoder = OneHotEncoder(inputCol=categoricalColumn+"Index", outputCol=categoricalColumn+"classVec")
  ## Add stages
  stages += [stringIndexer, encoder]

City
Marital
Race
Ethnicity
Gender
Suffix
County
smoker_status

In [7]:
assemblerInputs = categoricalColumnsclassVec + numericalColumns
assembler = VectorAssembler(inputCols = assemblerInputs, outputCol="features").setHandleInvalid("skip")
stages += [assembler]

In [8]:
prepPipeline = Pipeline().setStages(stages)
pipelineModel = prepPipeline.fit(dataset)
dataset = pipelineModel.transform(dataset)

In [9]:
# Note: 'TEST' is for validation while 'test' is for challenge submission
test = dataset.filter(col("dataset") == "test")
train, TEST = dataset.filter(col('dataset')=='train').randomSplit([0.7, 0.3], seed = 1337) 
# train = train.sample(False, .30, 1337)                    # Sample down for a more managable size
print("Number of rows in `train`: " + str(train.count()) + "\nNumber of rows in `test`: " + str(TEST.count()))

In [10]:
pipelineModel.write().overwrite().save("/mnt/data/ml/dayshospitalized/pipeline")
# display(dbutils.fs.ls("/mnt/data/ml/dayshospitalized/pipeline"))

### Linear Regression

In [12]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

In [13]:
# Using 'lm' for linear model and to avoid confusion with 'lr' for logistic regression

# Load training data
lm = LinearRegression(featuresCol = 'features', labelCol=label)

# Fit the model
lmModel = lm.fit(train)

# Summarize the model over the training set and print out some metrics
trainingSummary = lmModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)
trainingSummary.residuals.show()

Coefficients: [0.006335028053373022,-0.23703115945957912,0.10515520434516075,0.09637191021850106,0.15941075079555564,0.08522049348341842,-0.19740330367046283,-0.0016201297601181677,0.0565614743610255,-0.21257340797008098,0.30374066387693865,-0.16494436191203235,-0.24498799704867805,0.24320460316518083,-0.3761114569306748,-0.24306249433308497,0.04483291931634836,0.4754654065863743,-0.16142705026439375,-1.0973830332798291,-0.17472639885655475,-0.24243765291041527,0.2902724496470692,-0.23095616328369156,0.1136687269348201,-0.531518038772723,0.009937911800115695,0.052113703031923295,0.35002337430926367,0.946239833347187,0.14414145670897477,0.9672221122411436,0.7200406612623045,0.5805228061955341,-0.11304722660271797,-0.3613367194804903,0.2779483809716444,1.1656782195114783,-0.27032133915455864,-0.517843279500495,-0.10912467957666731,0.05064654987096732,0.08646801702585724,0.23151598496037934,0.047880938502438956,-0.6710958943124687,-0.3857209052277625,-0.5667536545404845,0.6630593504923239,0.633188178488543,0.9065673286386096,0.671262668611297,0.47032949050825185,-1.3828133653389811,-0.8122235016334829,-0.2686627147605846,-0.226169615063472,-0.4356937229665749,-0.6346597087422671,-0.47547677404890765,0.35425199823311015,-0.538337727748315,0.4278414160857043,0.373115968791843,0.453938535055354,0.13919995904623025,-0.8699935902256413,-1.241804510596283,-0.6629986770880054,0.21877477069427576,0.0644089698382091,-0.4637977290313876,-0.3726919322044846,1.0704983990030086,-0.9578583411675397,0.4068204295840562,0.18552216818624045,0.10763137485604152,-0.41126045543385054,-0.050063715835781335,-0.7435233692835247,-0.6665853900304956,0.7543988701187438,0.021618653270147576,-0.38903168668330546,0.07749225736963554,0.7874553537893232,0.36087167778635026,0.7342937187673387,0.10749200242119109,0.534744658566365,1.310172565759872,-0.39771242060536244,0.5951818648313291,0.26906059250816905,0.33226169659414834,-0.31941558288930216,-1.1179204843243165,0.514603822663474,-0.7942429321376088,-0.23890806682077384,0.06412173090837095,0.9361238302722551,0.5373708696221777,-0.4640569723082336,-0.02474483297601846,0.5941495179385679,-0.09206488576202625,0.7563965215196995,0.5348130051041,-0.8107954742972588,-0.6516192767062262,-0.8154203698482076,0.33042333709471133,-0.7906317300114514,0.027817148551300488,1.1880868803178415,-0.7055605738570866,-1.5245491502205424,0.20482681739378944,0.6146703635598777,-0.9494359124185106,-0.043557692991038045,-0.1449448968290569,0.5603269305061401,0.19631802232503467,0.7472655533139768,0.33982928129032747,0.03892319318291265,0.874360809074204,0.5613815263344047,2.0836069121839684,1.1421397221113285,-0.44702501603700745,-0.49839683187413386,-0.05867037454291951,1.1004176088180744,0.38453644417213956,-0.9979223574597007,-0.40044170956695824,-0.1560108944955798,-0.07322779401587437,-0.2362843069714546,-1.3219954728104852,-0.13175023273050393,0.545925212512281,0.6310452949326322,0.6353711228325003,0.5392113783031266,-0.22439844439943246,-0.3821308766033637,-0.36410729869204106,0.11023992340366606,0.0430383797736112,-0.16316488282135758,0.12608748104040668,-0.6387849439819979,0.8893582282426051,-0.6997358151445395,0.4877484979780307,2.335088726054656,-0.22930045459416412,-0.7747789599699713,0.5919588375082625,-0.7706899803299747,-1.2902662492244037,0.6868664322299716,-0.941193463297183,-1.4134170028103394,-0.6689988512048889,0.8059430018724707,-0.3115533253129361,0.17728999647182767,2.1728822738020708,-0.07131734869345464,-1.0351344304249457,-0.316202755534386,-1.3160127627751368,-0.443859087394923,-0.512029927169165,1.370054425730268,-0.6513325343779143,2.5463988563023667,0.6951384794311037,0.45929882582646214,0.8755271450497035,0.2702555420104755,0.669901263677323,0.36979557624872433,-0.9055376609010098,0.6584429673279979,-1.448749647068006,-0.9483711046230687,0.6220463384170967,-0.13278532002785828,1.0712685050631712,1.3530409357828121,1.7235618602512894,0.8103067542979951,0.6746073042558048,2.165712885102365,0.90571655757

In [14]:
# Create ParamGrid for Cross Validation
lmParamGrid = (ParamGridBuilder()
             .addGrid(lm.regParam, [0.01, 0.1, 0.5])
             .addGrid(lm.elasticNetParam, [0.0, 0.1, 0.25])
             .addGrid(lm.maxIter, [1, 5, 10])
             .build())

# Evaluate model
lmEvaluator = RegressionEvaluator(predictionCol="prediction", labelCol=label, metricName="rmse")

In [15]:
# Create 5-fold CrossValidator
lmCv = CrossValidator(estimator = lm,
                    estimatorParamMaps = lmParamGrid,
                    evaluator = lmEvaluator,
                    numFolds = 5)

# Run cross validations
lmCvModel = lmCv.fit(train)
lmCvModel.write().overwrite().save("/mnt/data/ml/dayshospitalized/lm")
print(lmCvModel)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
CrossValidatorModel_9890e609278b

In [16]:
# List important features
import pandas as pd

featurelist = pd.DataFrame(dataset.schema["features"].metadata["ml_attr"]["attrs"]["binary"]+dataset.schema["features"].metadata["ml_attr"]["attrs"]["numeric"]).sort_values("idx")
featurelist["Coefficient"] = pd.DataFrame(lmCvModel.bestModel.coefficients.toArray())
featurelist = sqlContext.createDataFrame(featurelist)

# display(featurelist)

idx,name,Coefficient
0,CityclassVec_Boston,0.0
1,CityclassVec_Worcester,-0.006890413213541185
2,CityclassVec_Springfield,0.0
3,CityclassVec_Lowell,0.0
4,CityclassVec_Cambridge,0.0
5,CityclassVec_Brockton,0.0
6,CityclassVec_New Bedford,0.0
7,CityclassVec_Quincy,0.0
8,CityclassVec_Lynn,0.0
9,CityclassVec_Newton,0.0


In [17]:
# Get Model Summary Statistics
lmCvSummary = lmCvModel.bestModel.summary
# print("Coefficient Standard Errors: " + str(lmCvSummary.coefficientStandardErrors))  # No s.e. avaialble for this model
# print("P Values: " + str(lmCvSummary.pValues)) # Last element is the intercept       # No p-values available either

# Use test set here so we can measure the accuracy of our model on new data
lmPredictions = lmCvModel.transform(TEST)
lmPredictions_test = lmCvModel.transform(test)   # (for submission)

# cvModel uses the best model found from the Cross Validation
# Evaluate best model
print('RMSE:', lmEvaluator.evaluate(lmPredictions))

RMSE: 4.56153070726048

### Poisson Regression
Poisson is a family within Generalized Linear Models for count dependent variables.

In [19]:
from pyspark.ml.regression import GeneralizedLinearRegression

In [20]:
# Does not like the number of variables. Only supports p<=4096.
glm = GeneralizedLinearRegression(family="poisson", featuresCol = 'features', labelCol=label)   # Time permitting, consider a sqrt link function

# Fit the model
glmModel = glm.fit(train)

Coefficients: [0.0057913677400165155,-0.01676188506302154,-0.02562526420205104,0.015360546553814974,0.03148177015178179,0.012760124003841144,-0.04869468382952927,-0.03469351907550611,-0.040539507056836566,-0.00955547862526301,0.02402197530172426,-0.03362965845745311,-0.09770495837038123,0.012587042609637746,0.037452428593696296,0.006280195423603079,0.0629002730301193,-0.018557605214569988,-0.033946809373355764,-0.07765101915553271,-0.04230613714550781,0.026290178094140648,0.0035422905518260332,-0.002428827606381591,0.015624323951025053,-0.016615754339620523,-0.05066188172496955,-0.02506335672545665,0.03911396778363197,0.10221640460401378,0.08184659174217614,0.08056642028409623,0.0283658461960551,0.006427220050885244,-0.10742783007906903,-0.05420983154478103,0.09016365588770577,0.025909400098497504,-0.014606181674015213,-0.0975804001037709,0.09333896937907654,0.05072228870956059,0.029092140999280418,-0.04482664518320156,0.027015545670500275,0.0564155837230567,0.05099075969512755,-0.10516431857668919,0.135818998310618,0.1561970554701371,0.13214207106111928,0.013390689576668481,-0.011152041360710569,-0.09327014797487886,-0.11584684266566754,0.05929700632474385,-0.08934760720974116,-0.0454844225553645,-0.1025121300055931,-0.004649402486602625,0.06333342097335418,-0.0831993212091497,0.06064483183888361,0.02493433650112664,0.1033751285512291,0.06263570578561245,0.04150073610137039,-0.10994363828590513,-0.11188077481865523,0.008047782213114127,0.04128216238161361,-0.07581936259581878,0.0008056878932644216,0.04936014613956438,-0.018110204166026784,0.09922563677780252,0.03581813320232653,-0.05531078867826566,-0.04382946667311749,-0.029773467181965066,-0.04782797691088521,-0.033249213031326726,-0.0016022931724307982,-0.022435461612515493,-0.12514815053452685,0.11713058089981077,0.19333963631717954,0.023899510133538953,-0.01655878617128521,0.06615584387883137,0.03192732869220485,0.1006769015104159,-0.08035000966078835,-0.0955619396605417,0.1034874720377032,-0.09460776715696669,0.15270933606495043,0.026837700389138134,0.053432173262079254,-0.00425811533756552,0.031215937196711854,-0.1693688932380936,-0.043140643825073995,0.13786169187426847,0.05363459121618766,0.05850912804444671,-0.013670622231056537,-0.017299164336940395,0.03296060506342943,-0.004292499452926316,-0.21113443099135032,-0.006951331125448473,-0.10813559277601471,0.057080893624042194,-0.0812368046812964,-0.06859400621758402,0.13901891652166587,0.020458547642866438,-0.20099775917197946,0.07013779138496855,-0.0005133146899734276,-0.18388355915911908,-0.012837400647657916,0.12369007051407863,-0.01163460039755267,0.10852230992450425,0.014077152529628016,0.03940223563947605,0.1031659643508902,0.13943371385682474,0.000790268010162916,0.07949765186865068,0.03030451756655848,-0.005537548027180165,-0.03346631513894024,0.02435165249309824,0.21996536684820495,0.10376360354404261,0.0401086303264877,0.028399183567102915,-0.04738989525589941,0.05112137133106687,-0.05717373046721544,-0.2239594267816344,-0.006859215518706745,0.08835987186748004,0.12258053865685546,0.10163140696498069,-0.0725984625675895,0.059204789723782764,-0.14474836117046308,-0.0753033956558834,-0.02163475717754244,-0.024368656892634618,0.07295935217674049,0.08556185120451751,-0.2034192715876672,0.10454185806847793,-0.127653192381704,0.038857479963387016,0.2549150186404912,0.027815623084277465,-0.06217701220927706,0.0017721283965175185,-0.04262344559839811,-0.1372850857177684,-0.123484880794085,0.1342844407582784,-0.07408659454012166,-0.1492513819467607,0.05338687552460674,0.0839005624532876,-0.0730379914634584,0.16659724768632056,-0.07386895651755586,-0.14664155935019924,0.11456378535267091,-0.1430154937868263,0.028379459298353026,-0.04007346187144295,0.05719965269285097,-0.03267250158630926,0.21036977694048936,0.09773811183450008,-0.12006542754429217,0.04805184878565733,0.056630777814314595,0.08440628026302362,0.14839517627367788,-0.10569053441603792,0.10160546220723073,-0.3969243054815839,-0.03342239123688241,-0.22100

In [21]:
# Summarize the model over the training set and print out some metrics
summary = glmModel.summary
# print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
# print("T Values: " + str(summary.tValues))
# print("P Values: " + str(summary.pValues))
print("Dispersion: " + str(summary.dispersion))
print("Null Deviance: " + str(summary.nullDeviance))
print("Residual Degree Of Freedom Null: " + str(summary.residualDegreeOfFreedomNull))
print("Deviance: " + str(summary.deviance))
print("Residual Degree Of Freedom: " + str(summary.residualDegreeOfFreedom))
print("AIC: " + str(summary.aic))
# print("Deviance Residuals: ")
summary.residuals().show()

Dispersion: 1.0
Null Deviance: 31226.002077215064
Residual Degree Of Freedom Null: 14414
Deviance: 24539.753470133564
Residual Degree Of Freedom: 13223
AIC: 88685.3466895799
Deviance Residuals: 
+--------------------+
 devianceResiduals|
+--------------------+
 -1.8885184618571225|
 -1.132324154857909|
 -0.8741403120585794|
 0.9098589473787951|
 -1.474042539300722|
 -0.4383016539781051|
 -1.0328506835928122|
 1.3490186476517692|
 -1.5457565832903288|
 -0.8270918399093663|
 -0.2313350761462869|
 -0.8866284186604781|
 0.1985633960073488|
-0.20653350733905998|
 -1.0746353073814843|
 0.4870912774916144|
 1.7238359451634824|
 -1.7870949185485823|
 1.0269933543293208|
 0.6215026311406067|
+--------------------+
only showing top 20 rows

In [22]:
# Create ParamGrid for Cross Validation
glmParamGrid = (ParamGridBuilder()
             .addGrid(glm.regParam, [0.01, 0.1, 0.5])
             .addGrid(glm.maxIter, [1, 5, 10])
             .build())

# Evaluate model
glmEvaluator = RegressionEvaluator(predictionCol="prediction", labelCol=label, metricName="rmse")

# Create 5-fold CrossValidator
glmCv = CrossValidator(estimator = glm,
                    estimatorParamMaps = glmParamGrid,
                    evaluator = glmEvaluator,
                    numFolds = 5)

# Run cross validations
glmCvModel = glmCv.fit(train)
glmCvModel.write().overwrite().save("/mnt/data/ml/dayshospitalized/glm")
print(glmCvModel)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
CrossValidatorModel_f24936ac5168

In [23]:
# List important features
featurelist = pd.DataFrame(dataset.schema["features"].metadata["ml_attr"]["attrs"]["binary"]+dataset.schema["features"].metadata["ml_attr"]["attrs"]["numeric"]).sort_values("idx")
featurelist["Coefficient"] = pd.DataFrame(glmCvModel.bestModel.coefficients.toArray())
featurelist = sqlContext.createDataFrame(featurelist)
# display(featurelist)

idx,name,Coefficient
0,CityclassVec_Boston,-1.0017390952763649E-4
1,CityclassVec_Worcester,2.8255179212766465E-4
2,CityclassVec_Springfield,-0.001117892704390284
3,CityclassVec_Lowell,-4.314107824808318E-4
4,CityclassVec_Cambridge,8.260834311321687E-4
5,CityclassVec_Brockton,7.487120139967952E-4
6,CityclassVec_New Bedford,-4.3333097954946807E-4
7,CityclassVec_Quincy,-4.2400133815810836E-4
8,CityclassVec_Lynn,-4.3312117861030793E-4
9,CityclassVec_Newton,-6.462998236405895E-4


In [24]:
# Get Model Summary Statistics
glmCvSummary = glmCvModel.bestModel.summary
# print("Coefficient Standard Errors: " + str(glmCvSummary.coefficientStandardErrors))
# print("P Values: " + str(glmCvSummary.pValues)) # Last element is the intercept

# Use test set here so we can measure the accuracy of our model on new data
glmPredictions = glmCvModel.transform(TEST)
# glmpredictions_test = glmcvModel.transform(test)

# cvModel uses the best model found from the Cross Validation
# Evaluate best model
print('RMSE:', glmEvaluator.evaluate(glmPredictions))

RMSE: 4.788142649159671

### Decision Tree Regression

In [26]:
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [27]:
# Load training data
dtr = DecisionTreeRegressor(labelCol=label, featuresCol="features")

In [28]:
# Create ParamGrid for Cross Validation
dtrParamGrid = (ParamGridBuilder()
             .addGrid(dtr.maxDepth, [2, 5, 10])
             .addGrid(dtr.maxBins, [10, 25, 50, 100])
             .build())

In [29]:
# Evaluate model
dtrEvaluator = RegressionEvaluator(labelCol=label, predictionCol="prediction", metricName="rmse")

In [30]:
# Create 5-fold CrossValidator
dtrCv = CrossValidator(estimator = dtr,
                      estimatorParamMaps = dtrParamGrid,
                      evaluator = dtrEvaluator,
                      numFolds = 5)

In [31]:
# Fit the model
dtrCvModel = dtrCv.fit(train)
dtrCvModel.write().overwrite().save("/mnt/data/ml/dayshospitalized/dtr")
print(dtrCvModel)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
CrossValidatorModel_42c38f709190

In [32]:
# Use test set here so we can measure the accuracy of our model on new data
dtrPredictions = dtrCvModel.transform(TEST)
dtrPredictions_test = dtrCvModel.transform(test)   # (for submission)

In [33]:
print('RMSE:', dtrEvaluator.evaluate(dtrPredictions))

RMSE: 4.5806782841214675

### Random Forest Regression

In [35]:
from pyspark.ml.regression import RandomForestRegressor

In [36]:
# Load training data
rfr = RandomForestRegressor(labelCol=label, featuresCol="features")

In [37]:
# Create ParamGrid for Cross Validation
rfrParamGrid = (ParamGridBuilder()
             .addGrid(rfr.maxDepth, [2, 5, 10])
             .addGrid(rfr.maxBins, [10, 25, 50, 100])
             .addGrid(rfr.numTrees, [5, 20, 50])
             .build())

In [38]:
# Evaluate model
rfrEvaluator = RegressionEvaluator(labelCol=label, predictionCol="prediction", metricName="rmse")

In [39]:
# Create 5-fold CrossValidator
rfrCv = CrossValidator(estimator = rfr,
                      estimatorParamMaps = rfrParamGrid,
                      evaluator = rfrEvaluator,
                      numFolds = 5)

In [40]:
# Fit the model
rfrCvModel = rfrCv.fit(train)
rfrCvModel.write().overwrite().save("/mnt/data/ml/dayshospitalized/rfr")
print(rfrCvModel)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.
CrossValidatorModel_d414e98e579f

In [41]:
# Use test set here so we can measure the accuracy of our model on new data
rfrPredictions = rfrCvModel.transform(TEST)
rfrPredictions_test = rfrCvModel.transform(test)    # (for submission)

In [42]:
print('RMSE:', rfrEvaluator.evaluate(rfrPredictions))

RMSE: 4.549101015190055

In [43]:
import pandas as pd
from pyspark.ml.tuning import CrossValidatorModel

rfrCvModel = CrossValidatorModel.load("/mnt/data/ml/dayshospitalized/rfr")

def ExtractFeatureImportance(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))
  
  
# ExtractFeatureImportance(model.stages[-1].featureImportances, dataset, "features")
dataset_fi = ExtractFeatureImportance(rfrCvModel.bestModel.featureImportances, train, "features")
dataset_fi = sqlContext.createDataFrame(dataset_fi)
display(dataset_fi)

idx,name,score
1171,age_years,0.19432391662313267
1038,Healthcare_Coverage,0.05176007596837511
562,Revenue,0.03973546087867221
885,QALY_Max,0.03598588568155739
932,QALY_Avg,0.020100348263069074
1151,2069-3_StdDev,0.018929850728438544
466,QALY_Min,0.017236232580133758
888,pct_covered,0.011261127730046753
572,49765-1_StdDev,0.011189517958408508
978,20565-8_StdDev,0.008323340796860618


In [44]:
from pyspark.ml.evaluation import RegressionEvaluator
rfrPredictions = rfrCvModel.transform(TEST)
# Evaluate model
rfrEvaluator = RegressionEvaluator(labelCol=label, predictionCol="prediction", metricName="mae")
rfrEvaluator.evaluate(rfrPredictions)


Out[17]: 3.681182293729037